# SVM + HOG 를 활용한 숫자(Handwritten Digit) 인식

**OpenCV**에서 제공하는 `digits.png` (5000개의 숫자 이미지)로부터 **HOG**(Histogram of Oriented Gradients) 특징을 추출하고, **SVM**으로 학습하여 **손글씨 숫자** 인식을 하는 과정을 다룹니다.

## 목차
1. **이론**
   - 1.1 kNN vs SVM 비교
   - 1.2 Deskew(이미지 기울임 보정)
   - 1.3 HOG 특징
2. **Colab 준비 & 라이브러리 설치**
3. **데이터 로드 (digits.png)**
4. **전처리** (deskew, hog)
5. **학습(SVM) & 예측**
6. **결과 & 정확도**
7. **과제** (파라미터 튜닝, 다른 기법 등)


# 1. 이론

## 1.1 kNN vs SVM
- **kNN**: 픽셀 자체를 feature vector로 사용 가능, 거리 기반 분류
- **SVM**: 고차원 특징(hog 등)을 사용하면 일반적으로 kNN보다 **일반화 성능**이 좋아질 수 있음

## 1.2 Deskew(이미지 기울임 보정)
- `cv2.moments`로 이미지의 2차 모멘트를 계산
- 기울어진 숫자 이미지를 warpAffine으로 0에 맞추어 정렬

수식:
$$ \text{skew} = \frac{m_{11}}{m_{02}} $$

이 skew 값을 이용해 변환행렬을 구성:
$$ M = \begin{bmatrix}1 & \text{skew} & -0.5 \times \text{SZ} \times \text{skew} \\ 0 & 1 & 0\end{bmatrix} $$
이후 `cv2.warpAffine(img, M, (SZ, SZ), ...)` 적용.

## 1.3 HOG (Histogram of Oriented Gradients)
- Sobel 미분으로 $gx, gy$를 구함
- 각 픽셀에서 $\text{magnitude}=\sqrt{gx^2 + gy^2}$, $\text{angle} = \arctan2(gy,gx)$
- angle을 **16개 bin**(0~15)로 quantize
- 이미지를 4개의 sub-region(10×10)으로 나누어, 각 sub-region에서 16-bin histogram
- 최종 4개의 sub-region × 16-bin = 64차원 특징 벡터

OpenCV에서는 **직접** HogDescriptor를 사용하기보단, 수동 구현 예시를 보여줍니다.

# 2. Colab 준비 & 라이브러리 설치

In [ ]:
!pip install opencv-python opencv-python-headless matplotlib numpy --quiet
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
print("OpenCV version:", cv.__version__)

OpenCV version: 4.10.0


## 2.1 digits.png 준비
OpenCV 저장소에서 `digits.png`를 다운로드하거나, Colab에 직접 업로드할 수 있습니다.
```python
# 아래 wget 명령으로 다운로드 가능 (OpenCV GitHub)
```


In [ ]:
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/samples/data/digits.png
import os
if not os.path.exists('digits.png'):
    print("digits.png download failed. Please upload manually.")
else:
    print("digits.png file is ready.")

digits.png file is ready.


# 3. 데이터 로드 (digits.png)
이미지 크기는 (100×50)의 셀(각 20×20)을 합친 1000×1000.
각 digit(0~9)가 5행씩 반복되어 총 50행.
각 행은 100열, digit당 20픽셀 폭으로 잘라쓰며, 5000개의 cell(각 digit 20×20).

In [ ]:
img = cv.imread('digits.png', cv.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError("digits.png not found.")
print("digits.png shape:", img.shape)

# 세로 50줄, 가로 100줄로 vsplit/hsplit
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
print("len(cells)=", len(cells), "  len(cells[0])=", len(cells[0]))

# train: 앞 50개 (열)  => x[:50]
# test:  뒷 50개 (열)  => x[50:]

train_cells = [ r[:50] for r in cells ]
test_cells  = [ r[50:] for r in cells ]

print("train_cells shape:", len(train_cells), "×", len(train_cells[0]))
print("test_cells shape:", len(test_cells),  "×", len(test_cells[0]))

digits.png shape: (1000, 2000)
len(cells)= 50   len(cells[0])= 100
train_cells shape: 50 × 50
test_cells shape: 50 × 50


# 4. 전처리: Deskew + HOG

## 4.1 Deskew 함수
- `cv.moments(img)`로 이미지 모멘트
- skew = mu11 / mu02
- warpAffine 으로 보정

In [ ]:
SZ = 20
affine_flags = cv.WARP_INVERSE_MAP | cv.INTER_LINEAR
bin_n = 16  # HOG bins

def deskew(img):
    m = cv.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew],[0,1,0]])
    img_out = cv.warpAffine(img, M, (SZ,SZ), flags=affine_flags)
    return img_out


## 4.2 HOG 함수
- Sobel($gx, gy$)
- magnitude & angle
- angle → 16개 bin
- 이미지(20×20)를 4 subregion(10×10)으로 쪼개어 histogram(16bin) 계산
- 최종 64차원 벡터

In [ ]:
def hog(img):
    gx = cv.Sobel(img, cv.CV_32F, 1, 0)
    gy = cv.Sobel(img, cv.CV_32F, 0, 1)
    mag, ang = cv.cartToPolar(gx, gy)

    bins = np.int32(bin_n * ang / (2*np.pi))
    # 20x20 -> 4 regions(10x10 each)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]

    hists = []
    for b, m in zip(bin_cells, mag_cells):
        hist_ = np.bincount(
            b.ravel(), weights=m.ravel(), minlength=bin_n
        )
        hists.append(hist_)

    hist = np.hstack(hists)  # shape: 64
    return hist


# 5. 학습(SVM) & 예측
- 각 digit(0~9)는 행(5행씩, 총 50행) × 열 100.
  - train: 앞 50열(=250개) → 총 2500개.
  - test: 뒤 50열 → 총 2500개.
- 전처리: Deskew 후 HOG
- SVM 파라미터(`C, gamma`)를 조정 가능.

OpenCV SVM
```python
svm = cv.ml.SVM_create()
svm.setKernel(cv.ml.SVM_LINEAR or cv.ml.SVM_RBF)
svm.setC(...)
svm.train(trainData, cv.ml.ROW_SAMPLE, responses)
svm.predict(testData)
```

In [ ]:
# (1) deskew + hog on train_cells
train_deskewed = []
for row in train_cells:
    deskewed_row = [deskew(cell) for cell in row]
    train_deskewed.append(deskewed_row)

train_hogdata = []
for row in train_deskewed:
    hog_row = [hog(cell) for cell in row]
    train_hogdata.append(hog_row)

trainData = np.float32(train_hogdata).reshape(-1, 64)
responses = np.repeat(np.arange(10), 250)[:, np.newaxis]  # 10 digit × 250 = 2500

# (2) SVM 설정
svm = cv.ml.SVM_create()
svm.setKernel(cv.ml.SVM_LINEAR)  # or RBF
svm.setType(cv.ml.SVM_C_SVC)
svm.setC(2.67)
svm.setGamma(5.383)

# (3) 학습
svm.train(trainData, cv.ml.ROW_SAMPLE, responses)
svm.save('svm_digits.dat')
print("SVM training complete.")

# (4) test
test_deskewed = []
for row in test_cells:
    deskewed_row = [deskew(cell) for cell in row]
    test_deskewed.append(deskewed_row)

test_hogdata = []
for row in test_deskewed:
    hog_row = [hog(cell) for cell in row]
    test_hogdata.append(hog_row)

testData = np.float32(test_hogdata).reshape(-1, 64)
ret, result = svm.predict(testData)

responses_test = np.repeat(np.arange(10), 250)[:, np.newaxis]
mask = (result == responses_test)
correct = np.count_nonzero(mask)
accuracy = correct*100.0 / result.size
print(f"Accuracy = {accuracy:.2f}%")


SVM training complete.
Accuracy = 93.80%


# 6. 결과 & 정확도
위 코드에서는 약 **94%** 정도의 정확도를 기대할 수 있습니다(파라미터에 따라 다름).
더 높은 정확도를 얻기 위해서는 **SVM 파라미터**(커널, C, gamma)를 더 튜닝하거나,
특징(HOG 세부 설정) 등을 개선할 수 있습니다.

# 7. 과제
1. **SVM 파라미터 튜닝**
   - kernel=RBF, C, gamma 등 여러 값을 시도하여 정확도 향상.
2. **HOG 파라미터** (bin_n, sub-region 분할) 변경.
3. **deskew** 절차 변경: resize or affine 변환 더 다양화.
4. **그레이스케일 이미지**(직접 PCA나 LDA로 변환) 후 SVM.
5. **데이터 증강**(rot, shift)으로 학습 데이터 늘리기.

### 과제 결과 예시: RBF Kernel
```python
svm = cv.ml.SVM_create()
svm.setKernel(cv.ml.SVM_RBF)
svm.setC(12.0)
svm.setGamma(0.5)
svm.train(trainData, cv.ml.ROW_SAMPLE, responses)
```
이런 식으로 **C, gamma**값을 바꿔가며 정확도를 확인해 보세요.

# Q&A
- **Q**: kNN vs SVM 중 어느 게 더 좋은가?
  - **A**: 일반적으로 SVM이 분류 성능이 나은 경우가 많고, kNN은 구현이 쉽지만 데이터 양이 많아질수록 느려질 수 있음.
- **Q**: digits.png 외에 다른 OCR 데이터에 적용하려면?
  - **A**: 동일한 deskew + hog 절차로 feature를 구하고, label에 맞춰 SVM 학습.
- **Q**: 정확도 94%면 더 올릴 수 없는가?
  - **A**: 파라미터 튜닝, data augmentation, 다른 특징(예: CNN) 등을 시도하면 더 높아질 수 있음.

이상으로 **SVM + HOG** 방식으로 **손글씨 숫자**(digits.png) 인식을 구현했습니다.
추가 과제를 통해 다양한 시도를 해보세요!